In [1]:
from keras import layers

In [2]:
from keras.models import Sequential
from keras.layers import Dense

In [3]:
model = Sequential()

model.add(Dense(32,input_dim = 784))
model.add(Dense(10 , activation='softmax'))

In [8]:
model.layers[0].get_weights() #获取层的权重
model.layers[0].set_weights(weights) #设置权重 shape必须相同

model.layers[0].get_config() #获取配置的详细信息

[array([[ 0.04300006,  0.0339147 , -0.03444719, ...,  0.0633276 ,
          0.05093934, -0.05984388],
        [-0.04292874,  0.07495474, -0.0763049 , ..., -0.08554589,
          0.04086911, -0.04507618],
        [ 0.00880529, -0.00379819, -0.0156771 , ...,  0.05367864,
          0.05900005,  0.00782073],
        ...,
        [ 0.05838151, -0.03405278, -0.03218485, ...,  0.02120452,
          0.03009786,  0.01550423],
        [-0.00374037, -0.01747674,  0.05625974, ...,  0.07175777,
         -0.03924921,  0.06642094],
        [ 0.07443877, -0.02073202, -0.04034144, ...,  0.01158834,
         -0.07829209,  0.03086971]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)]

In [9]:
save_config = model.layers[0].get_config() #获取配置的详细信息
#恢复层 method 1
layer = Dense.from_config(save_config)
#method 2
layer = layers.deserialize({'class_name':model.layers[0].__class__.__name__,
                            'config': save_config})

{'activation': 'linear',
 'activity_regularizer': None,
 'batch_input_shape': (None, 784),
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'dtype': 'float32',
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'dense_1',
 'trainable': True,
 'units': 32,
 'use_bias': True}

In [16]:
#一个输入 一个输出
model.layers[0].input
model.layers[0].output

model.layers[0].input_shape
model.layers[0].output_shape

(None, 32)

In [17]:
#multiple inputs and outputs
model.layers.get_input_at(index)
model.layers.get_output_at(index)

model.layers.get_input_shape_at(index)
model.layers.get_output_shape_at(index)

AttributeError: 'list' object has no attribute 'get_input_at'

In [ ]:
#regularizer 正则项

#constraint 约束项

In [ ]:
#Flatten layer 将数据一维化 不影响batch size

#Reshape layer 参数中不包含batch size信息

In [20]:
from keras.layers import Reshape

model = Sequential()
model.add( Reshape((3,4) , input_shape = (12 , )))

model.add( Reshape((6 , 2)) )

model.add( Reshape((-1 , 2 , 2)) )

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 3, 4)              0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 6, 2)              0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 3, 2, 2)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Permute layer 按照axis重新排列数据
#RNN 和 CNN对接时 使用
#广义转置
from keras.layers import Permute

#layer 参数 input_shape指定单个样本的shape
model = Sequential()
model.add(Permute((2,1) , input_shape=(10, 64)))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 64, 10)            0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [23]:
#RepeatVector layer 重复层
from keras.layers import RepeatVector

model = Sequential()
model.add(Dense(32 , input_dim = 32))

model.add(RepeatVector(3))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 32)             0         
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [26]:
#Lambda layer 对上一层输出结果施加一个运算操作
from keras.layers import Lambda
import keras.backend as K

def antirectifier(x):
    x -= K.mean(x , axis = 1 , keepdims=True)
    x = K.l2_normalize(x , axis = 1)
    
    pos = K.relu(x)
    neg = K.relu(-x)
    
    #将数据维度增大一倍
    return K.concatenate([pos , neg] , axis = 1)

def antirectifier_output_shape(input_shape):
    shape = list(input_shape)
    
    assert len(shape) == 3
    
    shape[-1] *= 2
    
    return tuple(shape)

model.add(Lambda(antirectifier , output_shape=antirectifier_output_shape))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 32)             0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 3, 64)             0         
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.add(Lambda(lambda x: x**2))
#每个元素去平方

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 32)             0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 3, 64)             0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 3, 64)             0         
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [30]:
#ActivityRegularizer layer 基于激活值更新损失函数值
from keras.layers import ActivityRegularization

model.add(ActivityRegularization())

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 32)             0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 3, 64)             0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 3, 64)             0         
_________________________________________________________________
activity_regularization_1 (A (None, 3, 64)             0         
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Masking layer 对输入序列信号进行屏蔽
from keras.layers import Masking
from keras.layers import LSTM

model = Sequential()
model.add(Masking(mask_value = 0.0 , input_shape(timesteps,features)))
model.add(LSTM(32))